In [37]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap, jvp, jit
import jax
from functools import partial

a = 1.4 # 2aB
nH = 2
atoms = ""
for i in range(nH):
    atoms += f"N {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="sto6g", unit='A', spin=0, verbose=4)
mol.build()

mf = scf.UHF(mol)
mf.kernel()

mo = mf.stability()[0]
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo = mf.stability()[0]
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)


nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()
et = mycc.ccsd_t()
print(mycc.e_tot + et)

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-37-generic', version='#37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Fri Jan 23 17:29:41 2026
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[ENV] PYSCF_EXT_PATH /home/sharmagroup/sharmagroup/pyscf-forge
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 14
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = A
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 N      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr 

In [47]:
# example for PT2
options = {'n_eql': 3,
           'n_prop_steps': 50,
            'n_ene_blocks': 1,
            'n_sr_blocks': 5,
            'n_blocks': 50,
            'n_walkers': 50,
            'seed': 2,
            'walker_type': 'uhf',
            'trial': 'uccsd_pt',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc.prop_unrestricted import prop_unrestricted
prop_unrestricted.prep_afqmc(mycc,options,chol_cut=1e-5)
# prop_unrestricted.run_afqmc(options,nproc=1)
option_file='options.bin'
import pickle
with open(option_file, 'wb') as f:
    pickle.dump(options, f)

#
# Preparing AFQMC calculation
# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver.
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (7, 7)
# Number of basis functions: 10
# Number of Cholesky vectors: 39
#


In [15]:
import numpy as np
from jax import random
from jax import numpy as jnp
from functools import partial
from ad_afqmc import config, stat_utils
from ad_afqmc.prop_unrestricted import prop_unrestricted, sampling
import time

In [49]:
config.setup_jax()
MPI = config.setup_comm()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

print = partial(print, flush=True)

ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ = (
    prop_unrestricted._prep_afqmc())

init_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

### initialize propagation
seed = options["seed"]
init_walkers = None
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(ham_data, prop, trial, wave_data)
h0 = ham_data['h0']

prop_data = prop.init_prop_data(trial, wave_data, ham_data, init_walkers)
if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
    raise ValueError(
        "Initial overlaps are zero. Pass walkers with non-zero overlap."
    )
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0

t, e0, te = trial.calc_energy_pt(prop_data['walkers'], ham_data, wave_data)

ept_sp = e0 + te - t*(e0-h0)
ept = jnp.array(jnp.sum(ept_sp) / prop.n_walkers)

print(ept)


# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 10
# nelec: (7, 7)
#
# n_eql: 3
# n_prop_steps: 50
# n_ene_blocks: 1
# n_sr_blocks: 5
# n_blocks: 50
# n_walkers: 50
# seed: 2
# walker_type: uhf
# trial: uccsd_pt
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#
-108.65569624578704


In [ ]:
import opt_einsum as oe

@partial(jit, static_argnums=0)
def _t1t2_walker_olp(
    self, walker_up: jax.Array, walker_dn: jax.Array, wave_data: dict
) -> complex:
    '''<HF|(t1+t2)|walker> = (t_ia G_ia + t_iajb G_iajb) * <HF|walker>'''
    noccA, t1A, t2AA = self.nelec[0], wave_data["t1a"], wave_data["t2aa"]
    noccB, t1B, t2BB = self.nelec[1], wave_data["t1b"], wave_data["t2bb"]
    t2AB = wave_data["t2ab"]
    # green_a = (walker_up.dot(jnp.linalg.inv(wave_data["mo_coeff"][0].T.conj() @ walker_up))).T
    # green_b = (walker_dn.dot(jnp.linalg.inv(wave_data["mo_coeff"][1].T.conj() @ walker_dn))).T
    green_a = (walker_up.dot(jnp.linalg.inv(walker_up[:noccA,:noccA]))).T
    green_b = (walker_dn.dot(jnp.linalg.inv(walker_dn[:noccB,:noccB]))).T
    green_a, green_b = green_a[:noccA, noccA:], green_b[:noccB, noccB:]
    o0 = self._calc_overlap(walker_up,walker_dn,wave_data)
    o1 = oe.contract("ia,ia", t1A, green_a, backend="jax") \
            + oe.contract("ia,ia", t1B, green_b, backend="jax")
    o2 = (
        0.5 * oe.contract("iajb, ia, jb", t2AA, green_a, green_a, backend="jax")
        + 0.5 * oe.contract("iajb, ia, jb", t2BB, green_b, green_b, backend="jax")
        + oe.contract("iajb, ia, jb", t2AB, green_a, green_b, backend="jax")
    )
    return (o1 + o2) * o0

@partial(jit, static_argnums=0)
def _t1t2_exp_chol(self, x: float, chol_i: jax.Array, walker_up: jax.Array,
                    walker_dn: jax.Array, wave_data: dict):
    '''
    unrestricted t_ia <psi_i^a|exp(x*h1_mod)|walker>/<HF|walker>
    '''
    walker_up_1x = walker_up + x * chol_i[0].dot(walker_up)
    walker_dn_1x = walker_dn + x * chol_i[1].dot(walker_dn)
    
    olp = _t1t2_walker_olp(self,walker_up_1x, walker_dn_1x, wave_data)
    o0 = self._calc_overlap(walker_up,walker_dn,wave_data)

    return olp/o0


In [41]:
prop_data['walkers'][0][0].shape

(10, 7)

In [42]:
walker_up = jnp.array(np.random.rand(10,7))
walker_dn = jnp.array(np.random.rand(10,7))

In [43]:
chol = ham_data["chol"].reshape(2, -1, trial.norb, trial.norb)
chol = chol.transpose(1,0,2,3)
chol_i = jnp.array(chol[0])

In [44]:
x = 0.0
f1 = lambda a: _t1t2_exp_chol(trial,a,chol_i,walker_up,walker_dn,wave_data)
_, fb_ad = jvp(f1, [x], [1.0])
print(fb_ad)

-34.28376263947549


In [45]:
def _calc_force_bias(
    self,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    ham_data: dict,
    wave_data: dict,
) -> jax.Array:
    """Calculates force bias < psi_T | chol_gamma | walker > / < psi_T | walker >"""
    nocc_a, ci1_a, ci2_aa = self.nelec[0], wave_data["t1a"], wave_data["t2aa"]
    nocc_b, ci1_b, ci2_bb = self.nelec[1], wave_data["t1b"], wave_data["t2bb"]
    ci2_ab = wave_data["t2ab"]
    green_a = (walker_up.dot(jnp.linalg.inv(walker_up[:nocc_a, :]))).T
    green_b = (walker_dn.dot(jnp.linalg.inv(walker_dn[:nocc_b, :]))).T
    green_occ_a = green_a[:, nocc_a:].copy()
    green_occ_b = green_b[:, nocc_b:].copy()
    greenp_a = jnp.vstack((green_occ_a, -jnp.eye(self.norb - nocc_a)))
    greenp_b = jnp.vstack((green_occ_b, -jnp.eye(self.norb - nocc_b)))

    chol_a = ham_data["chol"][0].reshape(-1, self.norb, self.norb)
    chol_b = ham_data["chol"][1].reshape(-1, self.norb, self.norb)
    rot_chol_a = chol_a[:, : self.nelec[0], :]
    rot_chol_b = chol_b[:, : self.nelec[1], :]
    lg_a = oe.contract("gpj,pj->g", rot_chol_a, green_a, backend="jax")
    lg_b = oe.contract("gpj,pj->g", rot_chol_b, green_b, backend="jax")
    lg = lg_a + lg_b

    # ref
    fb_0 = lg_a + lg_b

    # single excitations
    ci1g_a = oe.contract("pt,pt->", ci1_a, green_occ_a, backend="jax")
    ci1g_b = oe.contract("pt,pt->", ci1_b, green_occ_b, backend="jax")
    ci1g = ci1g_a + ci1g_b
    fb_1_1 = ci1g * lg
    ci1gp_a = oe.contract("pt,it->pi", ci1_a, greenp_a, backend="jax")
    ci1gp_b = oe.contract("pt,it->pi", ci1_b, greenp_b, backend="jax")
    gci1gp_a = oe.contract("pj,pi->ij", green_a, ci1gp_a, backend="jax")
    gci1gp_b = oe.contract("pj,pi->ij", green_b, ci1gp_b, backend="jax")
    fb_1_2 = -oe.contract(
        "gij,ij->g", chol_a, gci1gp_a, backend="jax")\
            - oe.contract("gij,ij->g", chol_b, gci1gp_b, backend="jax")
    fb_1 = fb_1_1 + fb_1_2

    # double excitations
    ci2g_a = oe.contract("ptqu,pt->qu", ci2_aa, green_occ_a, backend="jax")
    ci2g_b = oe.contract("ptqu,pt->qu", ci2_bb, green_occ_b, backend="jax")
    ci2g_ab_a = oe.contract("ptqu,qu->pt", ci2_ab, green_occ_b, backend="jax")
    ci2g_ab_b = oe.contract("ptqu,pt->qu", ci2_ab, green_occ_a, backend="jax")
    gci2g_a = 0.5 * oe.contract("qu,qu->", ci2g_a, green_occ_a, backend="jax")
    gci2g_b = 0.5 * oe.contract("qu,qu->", ci2g_b, green_occ_b, backend="jax")
    gci2g_ab = oe.contract("pt,pt->", ci2g_ab_a, green_occ_a, backend="jax")
    gci2g = gci2g_a + gci2g_b + gci2g_ab
    fb_2_1 = lg * gci2g
    ci2_green_a = (greenp_a @ (ci2g_a + ci2g_ab_a).T) @ green_a
    ci2_green_b = (greenp_b @ (ci2g_b + ci2g_ab_b).T) @ green_b
    fb_2_2_a = -oe.contract("gij,ij->g", chol_a, ci2_green_a, backend="jax")
    fb_2_2_b = -oe.contract("gij,ij->g", chol_b, ci2_green_b, backend="jax")
    fb_2_2 = fb_2_2_a + fb_2_2_b
    fb_2 = fb_2_1 + fb_2_2

    # overlap
    overlap_1 = ci1g
    overlap_2 = gci2g
    overlap = 1.0 + overlap_1 + overlap_2

    return (fb_0 + fb_1 + fb_2) / overlap

In [52]:
fb_tc = _calc_force_bias(
    trial,
    walker_up,
    walker_dn,
    ham_data,
    wave_data,
)
print(fb_tc)

[ 8.94829177e+00  5.78137016e+00  5.72488593e+00  1.29447737e+00
 -2.15157175e+00 -5.40136786e-01  4.04473133e-01  1.31797434e+00
  1.08760670e+00 -2.92221200e-02 -8.69773228e-01 -1.38945710e-01
  9.91910492e-01  4.25863688e-01 -1.03372025e-01 -4.98189222e-02
  6.83738106e-01  2.48028589e-01  4.95518826e-01  4.56164962e-01
 -7.04716326e-02 -3.78788078e-01 -8.38569923e-01  4.45358191e-01
 -5.06890965e-01  7.84922187e-03 -3.08658469e-01 -1.18006071e-01
 -1.37067191e-01  5.84124955e-02  9.70454368e-02  9.70519505e-02
  3.91864739e-03 -4.83767123e-02  6.34961686e-02 -2.21411319e-03
  3.67264572e-02 -1.10458120e-02 -9.33925900e-03]
